In [54]:
# IMPORT GENERIC PACKAGES
import numpy as np # numerical calc package
import pandas as pd # holds data
import matplotlib.pyplot as plt # plotting library
import seaborn as sns # pretty plotting
import pandas_profiling as pp
from xgboost import XGBClassifier
from sklearn.datasets import make_classification
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import MinMaxScaler

sns.set(rc={'figure.figsize':(20,10)})

from sklearn.model_selection import train_test_split # split dataset

from sklearn.linear_model import LogisticRegression # linear model for classification

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE

In [55]:
data = pd.read_csv('census.csv')

In [56]:
#report=pp.ProfileReport(data)

In [57]:
#report.to_file('data_assessment.html')

In [58]:
data.head(5)

,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


In [59]:
data.columns

Index(['age', 'workclass', 'education_level', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

In [60]:
# write list of features we want to use
features = ['age', 'workclass', 'education_level', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']

# Set X
X_temp = data[features]

# Convert text to numeric
X1 = pd.get_dummies(X_temp)

# Set y
y_temp = data["income"]
y_temp1 = pd.get_dummies(y_temp)

In [61]:
X = SelectKBest(chi2, k=8).fit_transform(X1, y_temp1)
X

array([[3.900e+01, 1.300e+01, 2.174e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [5.000e+01, 1.300e+01, 0.000e+00, ..., 1.000e+00, 0.000e+00,
        1.000e+00],
       [3.800e+01, 9.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [3.800e+01, 1.300e+01, 0.000e+00, ..., 1.000e+00, 0.000e+00,
        1.000e+00],
       [4.400e+01, 1.300e+01, 5.455e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [3.500e+01, 1.300e+01, 0.000e+00, ..., 1.000e+00, 0.000e+00,
        1.000e+00]])

In [62]:
pipeline = svm.SVC(kernel='linear')

In [63]:
Pipelining = Pipeline([('minmax', MinMaxScaler()), ('lr', LogisticRegression())])

In [64]:
y=y_temp1.drop('>50K',axis=1)
#print(y)

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 42)

In [66]:
Pipelining.fit(X_train,y_train)

C:\Users\grachelquinones\Anaconda3\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
         steps=[('minmax', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('lr',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [67]:
scores = Pipelining.score(X_test,y_test)
scores

0.839294598927525

In [68]:
# predictions
y_pred = Pipelining.predict(X_test)
print(y_pred)

[0 1 1 ... 1 1 1]


In [69]:
target_names = ["<=50K", ">50K"]
print(classification_report(y_test,y_pred,target_names=target_names))

              precision    recall  f1-score   support

       <=50K       0.72      0.57      0.63      4453
        >50K       0.87      0.93      0.90     13636

    accuracy                           0.84     18089
   macro avg       0.79      0.75      0.77     18089
weighted avg       0.83      0.84      0.83     18089

